In [0]:
# Loading the necessary packages
!pip install requests-html
from requests_html import HTMLSession
import numpy as np
import pandas as pd
import datetime
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

     |████████████████████████████████| 1.2MB 15.0MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for parse: filename=parse-1.15.0-cp36-none-any.whl size=23710 sha256=9749b7f301b63d0f109968f7b5540e517fdee4953fa2905e07d0cb20e89d713d
  Stored in directory: /root/.cache/pip/wheels/07/ee/c8/eced0759f09fc635398ab1b8e89c38549b28e5db7fd4a53ba5
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-cp36-none-any.whl size=78360 sha256=961358dff61f2954fc12f71707783a390d358990481fce11bb10e8603a983218
  Stored in directory: /root/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=cd462b5b3f858b1f2c773cc47bd336acf22e1e2db87051f53f3ced9d54f663f2
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built parse pyppeteer fake-useragent


'en_US.UTF-8'

In [0]:
# mounting my google drive to save files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# establish/open a session
session = HTMLSession()

In [0]:
# Set Site To Scrape
base_url = 'https://www.metacritic.com'

In [0]:
# initialise lists
album_details = []
genre_details = []
review_details = []
userscore_details = []
add_credits_list = []

# initialise album counter
tot_album_counter = 0

#~~~~~INPUTS~~~~~
# Select years to return data
start_year = 2000
end_year = 2019


year_range = range(start_year, end_year + 1)
for year in year_range:
    relative_url = '/browse/albums/score/metascore/year/filtered?sort=desc&year_selected='+str(year)
    print(f'~~~~Scraping Year: {year}~~~~')
    # submitting a GET request
    r = session.get(base_url+relative_url)
    print(f'Initial List GET Request Status: {r.status_code}')

    page_url_counter = len(r.html.find('div.pages li.page'))
    print(f'Number of pages for this year = {page_url_counter}')

    for page in range(0, page_url_counter):
        r = session.get(base_url+relative_url+'&page='+str(page))
        print(f'Primary List GET Request Status: {r.status_code}')
        
        #Finding all links within list on page for current year
        album_links = r.html.find('div.main_col div.product_wrap a')
        
        #Finding relative URLs on a page
        album_url = []
        if len(album_links) != None:
            for link in range(0,len(album_links)):
                album_url.append(base_url + album_links[link].attrs['href'])
        
        #Initialising printed counter to show page scraping progress
        page_album_counter = 0
        
        #Individual Album Details Scraping
        for album in album_url:
            s = session.get(album)
            print(f'Currently scraping {album}, status code: {s.status_code}')
            if s.status_code == 200:

                if s.html.find('div.product_title [itemprop=name]', first = True) != None:
                    album_name = s.html.find('div.product_title [itemprop=name]', first = True).text
                else:
                    album_name = ''
                if s.html.find('div.product_artist [itemprop=name]', first = True) != None:
                    p_artist_name = s.html.find('div.product_artist [itemprop=name]', first = True).text
                else:
                    p_artist_name = ''

                if s.html.find('li.product_credits a', first = True) != None:
                    add_artist_credits = s.html.find('li.product_credits a', first = True).text
                else:
                    add_artist_credits = ''
                add_artist_credits = add_artist_credits.split(', ')

                if s.html.find('li.product_company span.data',  first = True) != None:
                    label_name = s.html.find('li.product_company span.data',  first = True).text
                else:
                    label_name = ''

                if s.html.find('li.release span.data',  first = True) != None:
                    release_date = s.html.find('li.release span.data',  first = True).text
                    release_date = datetime.datetime.strftime((datetime.datetime.strptime(release_date,'%b %d, %Y')), '%Y-%m-%d')
                else:
                    release_date = ''
                
                #Scraping The Critic Review Details
                t = session.get(album+'/critic-reviews')
                print(f'    Scraping critic reviews, status code {t.status_code}')
                if t.status_code == 200:

                    if t.html.find('div.metascore_summary a.metascore_anchor span',  first = True) != None:
                        metascore = t.html.find('div.metascore_summary a.metascore_anchor span',  first = True).text
                    else:
                        metascore = ''

                    album_details.append((album_name,
                                        p_artist_name,
                                        label_name,
                                        release_date,
                                        metascore
                                        ))
                    album_details_df = pd.DataFrame(album_details, columns=['album_title','primary_artist', 'label', 'release_date','metascore'])

                    a = album_details_df.index[album_details_df['album_title'] == album_name].tolist()
                    b = album_details_df.index[album_details_df['primary_artist'] == p_artist_name].tolist()
                    c = album_details_df.index[album_details_df['release_date'] == release_date].tolist()

                    for i in a:
                        if i in a and i in b and i in c:
                            indexno = i

                    for artist in add_artist_credits:
                        if artist != '':
                            cred_list = [indexno, artist]
                            print(cred_list)
                            add_credits_list.append(cred_list)



                    if s.html.find('li.product_genre span.data',  first = True) != None:
                        genre = s.html.find('li.product_genre span.data',  first = True).text
                    else:
                        genre = ''
                    genre = genre.split(', ')

                    for g in genre:
                        genre_list = [indexno, g]
                        genre_details.append(genre_list)



                    if t.html.find('ol.critic_reviews li.critic_review') != None:
                        num_critic_reviews = len(t.html.find('ol.critic_reviews li.critic_review'))
                    else:
                        num_critic_reviews = 0
                    for i in range(0, num_critic_reviews):
                        if t.html.find('ol.critic_reviews li.critic_review')[i].find('div.source', first = True) != None:
                            reviewer = t.html.find('ol.critic_reviews li.critic_review')[i].find('div.source', first = True).text
                        else:
                            reviewer = None

                        if t.html.find('ol.critic_reviews li.critic_review')[i].find('div.date', first = True) != None:
                            review_date = t.html.find('ol.critic_reviews li.critic_review')[i].find('div.date', first = True).text
                            review_date = datetime.datetime.strftime((datetime.datetime.strptime(review_date,'%b %d, %Y')), '%Y-%m-%d')
                        else:
                            review_date = None

                        if t.html.find('ol.critic_reviews li.critic_review')[i].find('div.review_grade', first = True) != None:
                            review_score = t.html.find('ol.critic_reviews li.critic_review')[i].find('div.review_grade', first = True).text
                        else:
                            review_score = None

                        review_details.append((indexno, reviewer, review_date, int(review_score)))

                    #Scraping the user review data
                    u = session.get(album+'/user-reviews')
                    print(f'    Scraping user reviews, status code {u.status_code}')
                    if u.status_code == 200:

                        if u.html.find('div.userscore_details div.metascore_w', first = True) != None:
                            userscore = u.html.find('div.userscore_details div.metascore_w', first = True).text
                        else:
                            userscore = None
                        if userscore == 'tbd':
                          userscore = None
                        else:
                          userscore = float(userscore)
                        userscore_dist = ()

                        if len(u.html.find('div.score_details_module ol.score_counts span.count')) != 0:
                            for i in range(0,3):
                                if u.html.find('div.score_details_module ol.score_counts span.count')[i] != None:
                                    reviewseg = locale.atoi(u.html.find('div.score_details_module ol.score_counts span.count')[i].text)
                                else:
                                    reviewseg = None
                                userscore_dist += (reviewseg,)
                            tot_user_reviews = userscore_dist[0] + userscore_dist[1] + userscore_dist[2]
                        else:
                            userscore_dist = (None, None, None)
                            tot_user_reviews = None   

                        userscore_details.append((indexno, userscore, userscore_dist[0], userscore_dist[1], userscore_dist[2], tot_user_reviews))
            
            # Increase progress counters and print
            tot_album_counter = tot_album_counter + 1
            page_album_counter = page_album_counter + 1
            print(f'    Album complete. Finished {page_album_counter}/{len(album_links)} on page {page+1}/{page_url_counter} for {year}, {tot_album_counter} in total \n')

        #Creating the dataframes of the scraped data
        genre_df = pd.DataFrame(genre_details, columns = ['album_id','genre'])
        add_artist_credits_df = pd.DataFrame(add_credits_list, columns=['album_id','additional_artist'])
        userscore_details_df = pd.DataFrame(userscore_details, columns=['album_id','userscore', 'positive_reviews', 'mixed_reviews', 'negative_reviews','total_user_reviews'])
        review_details_df =  pd.DataFrame(review_details, columns=['album_id','reviewer', 'review_date', 'review_score'])
            
        #Output scraped data to csv
        album_details_df.to_csv('/content/gdrive/My Drive/album_detail_2015_2019.csv')
        add_artist_credits_df.to_csv('/content/gdrive/My Drive/additional_artist_2015_2019.csv')
        genre_df.to_csv('/content/gdrive/My Drive/genre_2015_2019.csv')
        review_details_df.to_csv('/content/gdrive/My Drive/review_detail_2015_2019.csv')
        userscore_details_df.to_csv('/content/gdrive/My Drive/userscore_detail_2015_2019.csv')

# Add permanent album ID column to album dataframe
album_details_df['album_id'] = album_details_df.index

Streaming output truncated to the last 5000 lines.
    Scraping critic reviews, status code 200
[1159, 'Roger Sellers']
    Scraping user reviews, status code 200
    Album complete. Finished 20/100 on page 5/8 for 2016, 1161 in total 

Currently scraping https://www.metacritic.com/music/infinite-summer/nzcalines, status code: 200
    Scraping critic reviews, status code 200
    Scraping user reviews, status code 200
    Album complete. Finished 21/100 on page 5/8 for 2016, 1162 in total 

Currently scraping https://www.metacritic.com/music/cautionary-tale/dylan-leblanc, status code: 200
    Scraping critic reviews, status code 200
    Scraping user reviews, status code 200
    Album complete. Finished 22/100 on page 5/8 for 2016, 1163 in total 

Currently scraping https://www.metacritic.com/music/the-small-hours/matt-berry, status code: 200
    Scraping critic reviews, status code 200
    Scraping user reviews, status code 200
    Album complete. Finished 23/100 on page 5/8 for 2016, 

In [0]:
metascore

''

In [0]:
#Creating the dataframes of the scraped data
genre_df = pd.DataFrame(genre_details, columns = ['album_id','genre'])
add_artist_credits_df = pd.DataFrame(add_credits_list, columns=['album_id','additional_artist'])
userscore_details_df = pd.DataFrame(userscore_details, columns=['album_id','userscore', 'positive_reviews', 'mixed_reviews', 'negative_reviews','total_user_reviews'])
review_details_df =  pd.DataFrame(review_details, columns=['album_id','reviewer', 'review_date', 'review_score'])

In [0]:
genre_df

In [0]:
add_artist_credits_df

In [0]:
userscore_details_df

In [0]:
review_details_df

In [0]:
album_details_df

In [0]:
#Output scraped data to csv
#album_details_df.to_csv('/content/gdrive/My Drive/album_detail_2010_2019.csv')
#add_artist_credits_df.to_csv('/content/gdrive/My Drive/additional_artist_2010_2019.csv')
#genre_df.to_csv('/content/gdrive/My Drive/genre_2010_2019.csv')
#review_details_df.to_csv('/content/gdrive/My Drive/review_detail_2010_2019.csv')
#userscore_details_df.to_csv('/content/gdrive/My Drive/userscore_details_2010_2019.csv')